<a href="https://colab.research.google.com/github/ethan1606/Plant_test1/blob/main/herb100s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Connect to GPU
select T4 GPU in runtime and connect

# 2. Upload dataset
upload labelled dataset into the folder as a data.zip file

Then unzip the file using the code below

In [1]:
#unzip the data.zip file into a folder named 'dataset'
!unzip /content/data.zip -d /content/dataset

Archive:  /content/data.zip
   creating: /content/dataset/images/
   creating: /content/dataset/labels/
  inflating: /content/dataset/classes.txt  
  inflating: /content/dataset/notes.json  
  inflating: /content/dataset/images/02b68df0-Pics-129.jpg  
  inflating: /content/dataset/images/030047da-Pics-46.jpg  
  inflating: /content/dataset/images/047f919f-Pics-103.jpg  
  inflating: /content/dataset/images/0fff76f0-Pics-112.jpg  
  inflating: /content/dataset/images/19c74eb0-Pics-144.jpg  
  inflating: /content/dataset/images/1b3c64f3-Pics-165.jpg  
  inflating: /content/dataset/images/1d2e5788-Pics-199.jpg  
  inflating: /content/dataset/images/1ec629c7-Pics-167.jpg  
  inflating: /content/dataset/images/2144dc61-Pics-114.jpg  
  inflating: /content/dataset/images/2416ff82-Pics-57.jpg  
  inflating: /content/dataset/images/27ed625c-Pics-108.jpg  
  inflating: /content/dataset/images/282cdfd4-Pics-77.jpg  
  inflating: /content/dataset/images/2cc03fbb-Pics-102.jpg  
  inflating: /conte

Note: the '!' allows shell commands to be run in the notebook rather than the terminal

# 3. Split the dataset into train and validation
To train a neural network the dataset must be split into training and validation images where during each epoch every image in the train folder is passed through the network performing backpropagation to adjust the weights and biases and then at the end of each epoch the images in the validation folder are used to check the model's performance

This will be done using a Python program from EdjeElectronics:

In [2]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# deploy the program specifiying where the dataset folder is and the split ratio of train to validation
!python train_val_split.py --datapath="/content/dataset" --train_pct=0.9

--2026-02-20 15:10:15--  https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3203 (3.1K) [text/plain]
Saving to: ‘/content/train_val_split.py’

/content/train_val_ 100%[===================>]   3.13K  --.-KB/s    in 0s      

2026-02-20 15:10:16 (57.0 MB/s) - ‘/content/train_val_split.py’ saved [3203/3203]

Created folder at /content/data/train/images.
Created folder at /content/data/train/labels.
Created folder at /content/data/validation/images.
Created folder at /content/data/validation/labels.
Number of image files: 101
Number of annotation files: 101
Images moving to train: 90
Images moving to validation: 11


# 4. Install Ultralytics
Ultralytics must be installed to use the yolo models

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.7 MB/s eta 0:00:00


# 5. Reformat dataset for Ultralytics
Ultralytics requires a specific file structure of the dataset with a YAML file

Again using code from EdjeElectronics to create the data.yaml file:

In [4]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/dataset/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
nc: 4
names:
- basil
- coriander
- mint
- parsley


#6 Train the YOLO model
Finally train with the chosen yolo model for a specified number of epochs

The best trained model will be saved to content/runs/detect/train/weights/best.pt and results can be found at content/runs/detect/train

In [5]:
#specify data path, model to use, number of epochs, and image size
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, i

# 7. Test the model
Now the model is trained we can use the images from the validation folder to see how it peforms

In [6]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
YOLO11s summary (fused): 101 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs

image 1/11 /content/data/validation/images/540a1afe-Pics-27.jpg: 384x640 1 coriander, 3 mints, 1 parsley, 55.1ms
image 2/11 /content/data/validation/images/6cb8abbb-Pics-96.jpg: 384x640 1 mint, 9.0ms
image 3/11 /content/data/validation/images/7998a3ed-Pics-75.jpg: 384x640 1 basil, 1 coriander, 1 mint, 8.4ms
image 4/11 /content/data/validation/images/8aed5c72-Pics-74.jpg: 384x640 1 basil, 1 coriander, 1 mint, 8.4ms
image 5/11 /content/data/validation/images/a82596b2-Pics-38.jpg: 384x640 1 basil, 2 parsleys, 8.4ms
image 6/11 /content/data/validation/images/a9d121ce-Pics-28.jpg: 384x640 1 basil, 4 corianders, 8.5ms
image 7/11 /content/data/validation/images/c0d459ae-Pics-110.jpg: 384x640 1 coriander, 8.4ms
image 8/11 /content/data/validation/images/c1557154-Pics-145.jpg: 384x640 2 corianders, 2 parsleys, 10.8ms
image 9/11 /

# 8. Zip model
Zip the detect folder to download the model and the results

In [7]:
# Create "model" folder to store model and results
!mkdir /content/model
!cp -r /content/runs/detect /content/model

# Zip into "model.zip"
%cd model
!zip -r /content/model.zip detect
%cd /content

/content/model
  adding: detect/ (stored 0%)
  adding: detect/train/ (stored 0%)
  adding: detect/train/args.yaml (deflated 53%)
  adding: detect/train/train_batch0.jpg (deflated 4%)
  adding: detect/train/results.csv (deflated 62%)
  adding: detect/train/BoxPR_curve.png (deflated 21%)
  adding: detect/train/confusion_matrix_normalized.png (deflated 26%)
  adding: detect/train/train_batch1.jpg (deflated 5%)
  adding: detect/train/weights/ (stored 0%)
  adding: detect/train/weights/last.pt (deflated 8%)
  adding: detect/train/weights/best.pt (deflated 8%)
  adding: detect/train/val_batch0_pred.jpg (deflated 7%)
  adding: detect/train/train_batch302.jpg (deflated 13%)
  adding: detect/train/val_batch0_labels.jpg (deflated 8%)
  adding: detect/train/train_batch301.jpg (deflated 10%)
  adding: detect/train/results.png (deflated 7%)
  adding: detect/train/BoxR_curve.png (deflated 11%)
  adding: detect/train/train_batch2.jpg (deflated 4%)
  adding: detect/train/confusion_matrix.png (deflated